## Práctica 3 - Deep Learning: Perceptrón Multicapa

Durante este primera apartado, vamos a empezar con un algoritmo generador de datos como aquel visto en la P-2. Este algoritmo será el encargo de crear sintéticamente la sample con la que vamos a entrenar a nuestro modelo de IA.

In [ ]:
def generate_people(n: int):
    # Iniciamos el dataset como una lista
    dataset = []
    for _ in range(n):
        # Creamos un dataelement como una lista
        dataelement = []
        # Presión arterial 
        dataelement.append(np.random.randint(MIN_ARTERIAL, MAX_ARTERIAL + 1))
        # Colesterol 
        dataelement.append(np.random.randint(MIN_COLESTEROL, MAX_COLESTEROL + 1))
        # Añadimos el dataelement al dataset
        dataset.append(dataelement)
    # Transformamos el dataset a un array de numpy
    dataset = np.array(dataset)
    return dataset
    
    
    def compute_probability_illness(dataset:np.array):
    arterial, colesterol, fc = dataset[:, 0], dataset[:, 1], dataset[:, 2]
        
    prob_arterial = np.where(arterial < NORMAL_ARTERIAL_LOW,  # Si la presión arterial es menor a la normal menor
                            (NORMAL_ARTERIAL_LOW - arterial) / (NORMAL_ARTERIAL_LOW - MIN_ARTERIAL), # El riesgo es el índice de desviación
                            np.where(arterial > NORMAL_ARTERIAL_HIGH, # Si es mayor a la normal mayor
                                     (arterial - NORMAL_ARTERIAL_HIGH) / (MAX_ARTERIAL - NORMAL_ARTERIAL_HIGH), # El riesgo es el índice de desviación
                                     0)) # Si no es ni menor a la normal menor ni mayor a normal mayor, el riesgo es 0
    
    prob_colesterol = np.where(colesterol > NORMAL_COLESTEROL_HIGH, # Si el colesterol es mayor al máximo normal
                               (colesterol - NORMAL_COLESTEROL_HIGH) / (MAX_COLESTEROL - NORMAL_COLESTEROL_HIGH), # El riesgo es el índice de desviación
                               0) # Si no el riesgo es 0

    
    # Todas las probabilidades ostentan un mismo peso que suman como máximo 1
    total_prob = 1/2 * prob_colesterol + 1/2 * prob_arterial
    
    # Devolvemos el dataset añadiendole una nueva columna
    return np.column_stack((dataset, total_prob))


# Funcion que aplica cota sobre datos
def apply_threshold(threshold:float, dataset:np.array):
    risk_class = np.where(dataset[:, -1] > threshold, 1, 0)
    return np.column_stack((dataset, risk_class))

***
**Jaime Alonso Fernández**  
**Universidad Complutense de Madrid**  
**Grado en Ingeniería del Software (Plan 2019)**  
**Asignatura de Aprendizaje Automático y Big Data**
***

